In [1]:
# autoreload
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys

import pathmagic
from tools.project import proj_dir

In [ ]:
"""
import torch
from net.pool_net import PoolNet, PoolNetEncoderBlock, PoolNetDecoderBlock

batch_size = 4
num_frames = 5
num_channels = 3
w, h = 224, 224

x = torch.rand(batch_size, num_frames, num_channels, w, h)

net = PoolNet(post_conv_channel_size=1, bn_eps=1e-3, last_relu=False, bias=False)

out = net(x)

#block1 = PoolNetEncoderBlock(num_channels, 10, False)
#block2 = PoolNetEncoderBlock(10, 20)

#block_dec = PoolNetDecoderBlock(10, 1)

#x_out, x_pool_out = block1(x, None)
#x_dec, x_pool_dec = block_dec(x_out, x_pool_out)
#x_out2, x_pool_out2 = block2(x_out, x_pool_out)
"""

In [ ]:
out.size()

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

img1 = torch.Tensor(np.asarray(Image.open("/home/kevin/Pictures/img1.png").convert('RGB'))) / 255
img2 = torch.Tensor(np.asarray(Image.open("/home/kevin/Pictures/img2.png").convert('RGB'))) / 255

img1 = img1[:120, :120].permute(2, 0, 1)
img2 = img2[:120, :120].permute(2, 0, 1)

x = torch.stack([img1, img2]).unsqueeze(0)

#x = torch.cat([x, 0000*torch.rand(x.size())])

print(x.size())

y = torch.max(x, dim=1)[0]

plt.imshow(y[0].permute(1, 2, 0))

In [20]:
import torch
from net.pool_net import PoolNet, PoolNetEncoder, PoolNetDecoder
from tools.tools import module_flat_str, deterministic

deterministic()
    
net = PoolNet(enc_sizes=PoolNetEncoder.dft_sizes * 1 // 2,
              dec_sizes=PoolNetDecoder.dft_sizes * 1 // 2,
              bn_eps=1e-4,
              bias=False,
              act_func=torch.nn.ReLU(),
              enc_pool_layers=None,
              dec_pool_layers=False,
              use_enc_cc=False
             )

#print(module_flat_str(net, module_filter=[torch.nn.ReLU, torch.nn.BatchNorm2d]))
print(net.flat_str(module_filter=[torch.nn.ReLU, torch.nn.BatchNorm2d]))

batch_size = 4
num_frames = 5
num_channels = 3
w, h = 112, 112
#x = torch.rand(batch_size, num_frames, num_channels, w, h)

with torch.no_grad():
    out = net(x)
    print((out**2).sum())
    print(out.size())

Encoder:
Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
GlobalMaxPool()
Conv2d(64, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(48, 48, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
GlobalMaxPool()
Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
GlobalMaxPool()
Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
GlobalMaxPool()
Conv2d(256, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(192, 192, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)

Decoder:
Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
ConvTranspose2d(192, 192, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
ConvTranspose2d(192, 12

In [26]:
import torch
from net.pool_net import PoolNet, PoolNetEncoder, PoolNetDecoder
from net.coc_depth_net import CoCDepthEncShareNet
from tools.tools import module_flat_str, deterministic

deterministic()
    
net = CoCDepthEncShareNet(enc_sizes=PoolNetEncoder.dft_sizes * 1 // 2,
                  dec_sizes=PoolNetDecoder.dft_sizes * 1 // 2,
                  bn_eps=1e-4,
                  bias=False,
                  act_func=torch.nn.ReLU(),
                  enc_pool_layers=None,
                  dec_pool_layers=False,
                  use_coc_cc=True, use_depth_cc=True,
                 )

print(net.flat_str(module_filter=[torch.nn.ReLU, torch.nn.BatchNorm2d]))

#print(net)
#print(module_flat_str(net, module_filter=[torch.nn.ReLU, torch.nn.BatchNorm2d]))
#print(net.flat_str(module_filter=[torch.nn.ReLU, torch.nn.BatchNorm2d]))

batch_size = 4
num_frames = 5
num_channels = 3
w, h = 112, 112

with torch.no_grad():
    x = torch.rand(batch_size, num_frames, num_channels, w, h)
    x_coc, x_depth = net(x)
    print(x_coc.size(), x_depth.size())

Encoder:
Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
GlobalMaxPool()
Conv2d(64, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(48, 48, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
GlobalMaxPool()
Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
GlobalMaxPool()
Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
GlobalMaxPool()
Conv2d(256, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(192, 192, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)

Decoder CoC:
Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
ConvTranspose2d(192, 192, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
Conv2d(320, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(192, 192, kernel_size=(3, 3)